In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1pY8Rs7KRjoC2Pb47AJgBKPOs1_P2tLpH'})
download.GetContentFile('dataset.tar')

In [6]:
!ls

adc.json  dataset.tar  sample_data


In [0]:
!tar -xvf dataset.tar

In [0]:
os.listdir('dataset')

In [9]:
!ls

adc.json  dataset  dataset.tar	sample_data


In [10]:
import os
import cv2
import numpy as np
import pickle
from sklearn.model_selection import train_test_split


def get_images_labels():
    images = []
    labels = []
    for label in os.listdir('dataset'):
        for i in range(2400):
            img = cv2.imread('./dataset/'+ label + '/{}.jpg'.format(i+1), 0)
            images.append(np.array(img, dtype=np.uint8))
            labels.append(label)

    return images, labels

images, labels = get_images_labels()
images = np.array(images, dtype="float")
labels = np.array(labels)
print('Size of images', len(images))
print('Size of labels', len(labels))
(Xtrain, Xtest, Ytrain, Ytest) = train_test_split(images, labels, test_size=0.3, random_state=42)
print('Size of Xtrain', Xtrain.shape)
print('Size of Xtest', Xtest.shape)
print('Size of Ytrain', Ytrain.shape)
print('Size of Ytest', Ytest.shape)



Size of images 105600
Size of labels 105600
Size of Xtrain (73920, 50, 50)
Size of Xtest (31680, 50, 50)
Size of Ytrain (73920,)
Size of Ytest (31680,)


In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.utils import plot_model
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [12]:
# fix random seed
np.random.seed(7)
num_pixels = Xtrain.shape[1] * Xtrain.shape[2]
print(num_pixels)
#flattening the image for training
Xtrain = Xtrain.reshape(Xtrain.shape[0], 1, 50, 50).astype('float32')
Xtest = Xtest.reshape(Xtest.shape[0], 1, 50, 50).astype('float32')
#normalizing input
Xtrain /= 255
Xtest /= 255
#setting the output
Ytrain = np_utils.to_categorical(Ytrain)
Ytest = np_utils.to_categorical(Ytest)
print(Ytrain.shape)
print(Ytest.shape)
num_classes = Ytest.shape[1]

print(num_classes)

2500
(73920, 44)
(31680, 44)
44


In [0]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=5,input_shape=(1, 50, 50), activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size=3,activation = 'relu'))
    model.add(Conv2D(64, kernel_size=3,activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size=3, activation = 'relu'))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Dense(128, activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Dense(44, activation = "softmax"))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [14]:
model = cnn_model()
model.fit(Xtrain, Ytrain, validation_data=(Xtest, Ytest), epochs=40, batch_size=512, verbose=2)
scores = model.evaluate(Xtest, Ytest, verbose=0)
print('CNN Error {}'.format(100-scores[1]*100))

Train on 73920 samples, validate on 31680 samples
Epoch 1/10
 - 24s - loss: 1.0683 - acc: 0.6999 - val_loss: 0.0199 - val_acc: 0.9949
Epoch 2/10
 - 21s - loss: 0.0696 - acc: 0.9775 - val_loss: 0.0029 - val_acc: 0.9992
Epoch 3/10
 - 21s - loss: 0.0309 - acc: 0.9905 - val_loss: 0.0011 - val_acc: 0.9996
Epoch 4/10
 - 21s - loss: 0.0215 - acc: 0.9937 - val_loss: 7.1545e-04 - val_acc: 0.9997
Epoch 5/10
 - 21s - loss: 0.0177 - acc: 0.9947 - val_loss: 5.4406e-04 - val_acc: 0.9997
Epoch 6/10
 - 21s - loss: 0.0140 - acc: 0.9960 - val_loss: 1.7302e-04 - val_acc: 0.9999
Epoch 7/10
 - 21s - loss: 0.0126 - acc: 0.9958 - val_loss: 1.1202e-04 - val_acc: 0.9999
Epoch 8/10
 - 21s - loss: 0.0127 - acc: 0.9963 - val_loss: 5.7814e-05 - val_acc: 1.0000
Epoch 9/10
 - 21s - loss: 0.0121 - acc: 0.9964 - val_loss: 1.6899e-04 - val_acc: 0.9999
Epoch 10/10
 - 21s - loss: 0.0091 - acc: 0.9973 - val_loss: 2.0074e-04 - val_acc: 0.9999
CNN Error 0.006313131313135045


In [20]:
model.summary()
model.save('data.h5')
print('Model is being saved')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 46, 46)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 23, 23)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 23, 23)        92        
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 23, 23)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 21, 21)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 19, 19)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 9, 9)          0         
__________

In [17]:
!ls


adc.json  data.h5  dataset  dataset.tar  sample_data
